In [788]:
%%capture
!pip install category_encoders==2.*
!pip install pandas_profiling==2.*

In [789]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from category_encoders import OneHotEncoder, OrdinalEncoder

import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

from pandas_profiling import ProfileReport

import pickle


Replace the file paths with downloaded files fromt the repository.

In [790]:
#df_1 = pd.read_csv(r"C:\Users\wscot\OneDrive\Desktop\Kickstarter_2021-12-09T03_20_13_242Z\Kickstarter.csv")
#df_2 = pd.read_csv(r"C:\Users\wscot\OneDrive\Desktop\Kickstarter_2021-12-09T03_20_13_242Z\Kickstarter001.csv")
#df_3 = pd.read_csv(r"C:\Users\wscot\OneDrive\Desktop\Kickstarter_2021-12-09T03_20_13_242Z\Kickstarter002.csv")
#df_5 = pd.read_csv(r"C:\Users\wscot\OneDrive\Desktop\Kickstarter_2021-12-09T03_20_13_242Z\Kickstarter004.csv")
#df_6 = pd.read_csv(r"C:\Users\wscot\OneDrive\Desktop\Kickstarter_2021-12-09T03_20_13_242Z\Kickstarter005.csv")
#df_7 = pd.read_csv(r"C:\Users\wscot\OneDrive\Desktop\Kickstarter_2021-12-09T03_20_13_242Z\Kickstarter006.csv")
#df_8 = pd.read_csv(r"C:\Users\wscot\OneDrive\Desktop\Kickstarter_2021-12-09T03_20_13_242Z\Kickstarter007.csv")

In [791]:
def data_cleaner(df):
    
    
# Remove the high cardinatilty columns    
    threshold = 500
    high_card_cols = [col for col in df.select_dtypes('object')
                        if df[col].nunique() > threshold]
    df = df.drop(high_card_cols, axis=1)
    
    
# Remove other columns that I don't know what to do with
    df.drop(['category'], axis=1, inplace=True)
    df.drop(['country'], axis=1, inplace=True)
    df.drop(['currency'], axis=1, inplace=True)
    df.drop(['currency_symbol'], axis=1, inplace=True)
    df.drop(['source_url'], axis=1, inplace=True)
    df.drop(['state_changed_at'], axis=1, inplace=True)
    df.drop(['static_usd_rate'], axis=1, inplace=True)
    df.drop(['usd_type'], axis=1, inplace=True)
    df.drop(['current_currency'], axis=1, inplace=True)
    df.drop(['created_at'], axis=1, inplace=True)
    df.drop(['deadline'], axis=1, inplace=True)
    df.drop(['id'], axis=1, inplace=True)
    df.drop(['is_starrable'], axis=1, inplace=True)
    df.drop(['disable_communication'], axis=1, inplace=True)
    df.drop(['launched_at'], axis=1, inplace=True)
    df.drop(['usd_exchange_rate'], axis=1, inplace=True)
    df.drop(['fx_rate'], axis=1, inplace=True)
    df.drop(['currency_trailing_code'], axis=1, inplace=True)
    df.drop(['staff_pick'], axis=1, inplace=True)
    df.drop(['usd_pledged'], axis=1, inplace=True)
    df.drop(['pledged'], axis=1, inplace=True)
    df.drop(['spotlight'], axis=1, inplace=True)
    
    

    
    df['state'] = df['state'].replace('failed', 0, regex=True)
    df['state'] = df['state'].replace('successful', 1, regex=True)
    
    df.drop(df.loc[df['state'] == 'canceled'].index, inplace=True)
    df.drop(df.loc[df['state'] == 'live'].index, inplace=True)

    
    return df
    

In [792]:
frames = [df_1, df_2, df_3,df_5,df_6,df_7,df_8]

df = pd.concat(frames)

In [793]:
df = data_cleaner(df)

In [794]:
#profile = ProfileReport(df)
#profile

In [795]:
#Split the data into Feeature Matrix and Target Vector
target = 'state'
y = df[target]
X = df.drop(columns=target)


In [796]:
y=y.astype('int')


In [797]:
#train test splits
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [798]:
# Establish a baseline

baseline_acc = y_train.value_counts(normalize=True).max()
print('Baseline accuracy:', baseline_acc)

Baseline accuracy: 0.6611048794167134


In [799]:
# create and fit the model
model_boost = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    XGBClassifier()
)

model_boost.fit(X_train, y_train)

[12:14:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\wscot\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['country_displayable_name'],
                                mapping=[{'col': 'country_displayable_name',
                                          'data_type': dtype('O'),
                                          'mapping': the United States      1
the United Kingdom     2
Canada                 3
the Netherlands        4
Germany                5
France                 6
Japan                  7
Spain                  8
Sweden                 9
Poland                10
Australia             11
Mexico                12
Austria               13
Italy                 14
Hong Kong             15
Denmark               16
Norway                17
New Zealand           18
Singapore             19
Switzerla...
                               gamma=0, gpu_id=-1, importance_type=None,
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
     

In [800]:
# create and fit the model
model_bag = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier()
)
model_bag.fit(X_train, y_train);

In [801]:
from sklearn.tree import DecisionTreeClassifier, plot_tree
# Decision Tree
model_dt = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(strategy='mean'),
    DecisionTreeClassifier(random_state=42)
)

model_dt.fit(X_train, y_train)

Pipeline(steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['country_displayable_name'],
                                mapping=[{'col': 'country_displayable_name',
                                          'data_type': dtype('O'),
                                          'mapping': the United States      1
the United Kingdom     2
Canada                 3
the Netherlands        4
Germany                5
France                 6
Japan                  7
Spain                  8
Sweden                 9
Poland                10
Australia             11
Mexico                12
Austria               13
Italy                 14
Hong Kong             15
Denmark               16
Norway                17
New Zealand           18
Singapore             19
Switzerland           20
Belgium               21
Ireland               22
Luxembourg            23
Slovenia              24
Greece                25
NaN                   -2
dtype: int64}])),
                ('simpleimputer',

In [802]:
print('Bagging Model')
print('Training accuracy:', model_bag.score(X_train, y_train))
print('Validation accuracy:', model_bag.score(X_val, y_val))
print()
print('Boosting Model')
print('Training accuracy:', model_boost.score(X_train, y_train))
print('Validation accuracy:', model_boost.score(X_val, y_val))
print()
print('Decision Tree Classifier')
print('Training accuracy:', model_dt.score(X_train, y_train))
print('Validation accuracy:', model_dt.score(X_val, y_val))

Bagging Model
Training accuracy: 1.0
Validation accuracy: 0.9873808188446439

Boosting Model
Training accuracy: 0.9999298934380259
Validation accuracy: 0.9927089175546832

Decision Tree Classifier
Training accuracy: 1.0
Validation accuracy: 0.982893998878295


In [803]:
global model_boost
pickle.dump(model_boost, open('model.pickle', 'wb'))

In [804]:
filename = 'finalized_model.sav'
pickle.dump(model_boost, open(filename, 'wb'))

In [805]:
from category_encoders import *
features = model_boost.named_steps['ordinalencoder'].get_feature_names()
gini = model_boost.named_steps['xgbclassifier'].feature_importances_ 
pd.Series(data=gini, index=features).sort_values().tail(10).plot(kind='barh')
plt.ylabel('features')
plt.xlabel('gini importance')

Text(0.5, 0, 'gini importance')

In [806]:
from sklearn.metrics import plot_confusion_matrix, classification_report
plot_confusion_matrix(
    model_boost,
    X_val,
    y_val,
    values_format = '.0f'  
)

plt.show()

In [807]:
print(classification_report(y_val,
                      model_boost.predict(X_val),
                      target_names =['not successful','successful']))

                precision    recall  f1-score   support

not successful       1.00      0.98      0.99      1207
    successful       0.99      1.00      0.99      2359

      accuracy                           0.99      3566
     macro avg       0.99      0.99      0.99      3566
  weighted avg       0.99      0.99      0.99      3566

